In [1]:
# 先导入数据集grilic.dta
library(haven)
grilic <- read_dta("stata_data/grilic.dta")

In [2]:
grilic 

# A tibble: 758 × 20
     rns rns80   mrt mrt80  smsa smsa80   med    iq   kww  year   age age80
   <dbl> <dbl> <dbl> <dbl> <dbl>  <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1     0     0     0     1     1      1     8    93    35    68    19    31
 2     0     0     0     1     1      1    14   119    41    66    23    37
 3     0     0     0     1     1      1    14   108    46    67    20    33
 4     0     0     0     1     1      1    12    96    32    66    18    32
 5     0     0     1     1     1      1     6    74    27    73    26    34
 6     0     0     0     0     1      1     8    91    24    66    16    30
 7     0     0     1     1     1      1     8   114    50    73    30    38
 8     0     0     1     1     0      0    14   111    37    67    23    36
 9     0     0     1     1     1      1    12    95    44    66    22    36
10     0     0     1     1     0      0    13   132    44    73    30    38
# ℹ 748 more rows
# ℹ 8 more variables: s <dbl>, s80 <dbl>, expr <d

In [5]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the ]8;;http://conflicted.r-lib.org/conflicted package]8;; to force all conflicts to become errors


In [6]:
summary(grilic)

      rns             rns80             mrt             mrt80       
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:1.0000  
 Median :0.0000   Median :0.0000   Median :1.0000   Median :1.0000  
 Mean   :0.2691   Mean   :0.2929   Mean   :0.5145   Mean   :0.8984  
 3rd Qu.:1.0000   3rd Qu.:1.0000   3rd Qu.:1.0000   3rd Qu.:1.0000  
 Max.   :1.0000   Max.   :1.0000   Max.   :1.0000   Max.   :1.0000  
      smsa            smsa80            med              iq        
 Min.   :0.0000   Min.   :0.0000   Min.   : 0.00   Min.   : 54.00  
 1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.: 9.00   1st Qu.: 95.25  
 Median :1.0000   Median :1.0000   Median :12.00   Median :104.00  
 Mean   :0.7045   Mean   :0.7124   Mean   :10.91   Mean   :103.86  
 3rd Qu.:1.0000   3rd Qu.:1.0000   3rd Qu.:12.00   3rd Qu.:113.75  
 Max.   :1.0000   Max.   :1.0000   Max.   :18.00   Max.   :145.00  
      kww             year            age

# 普通OLS

先考虑用普通ols，看看效果：

In [12]:
# 先进行普通OLS，作为对比
fit <- lm(lw ~ s + expr + tenure + rns + smsa, data = grilic)
result <- summary(fit)
summary(fit)


Call:
lm(formula = lw ~ s + expr + tenure + rns + smsa, data = grilic)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.11684 -0.22626 -0.01511  0.23103  1.23738 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.103675   0.085097  48.223  < 2e-16 ***
s            0.102643   0.005849  17.549  < 2e-16 ***
expr         0.038119   0.006327   6.025 2.65e-09 ***
tenure       0.035615   0.007742   4.600 4.96e-06 ***
rns         -0.084080   0.028797  -2.920  0.00361 ** 
smsa         0.139667   0.028082   4.974 8.15e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3464 on 752 degrees of freedom
Multiple R-squared:  0.3521,	Adjusted R-squared:  0.3478 
F-statistic: 81.75 on 5 and 752 DF,  p-value: < 2.2e-16


In [13]:
library(sandwich)
robust_se <- vcovHC(fit, type = "HC")

library(lmtest)
coeftest(fit, vcov = robust_se)


t test of coefficients:

              Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  4.1036750  0.0873188 46.9965 < 2.2e-16 ***
s            0.1026430  0.0061852 16.5948 < 2.2e-16 ***
expr         0.0381189  0.0065882  5.7860 1.058e-08 ***
tenure       0.0356146  0.0079671  4.4702 9.019e-06 ***
rns         -0.0840797  0.0294158 -2.8583  0.004377 ** 
smsa         0.1396666  0.0279447  4.9980 7.213e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


教育回报率过高，考虑是不是有内生性的问题。

下面考虑引入能力的代理变量iq进行回归，相当于加入遗漏变量：

In [14]:
# 加入替代变量iq
fit <- lm(lw ~ s + expr + tenure + rns + smsa + iq, data = grilic)
result <- summary(fit)
summary(fit)


Call:
lm(formula = lw ~ s + expr + tenure + rns + smsa + iq, data = grilic)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.16056 -0.21786 -0.00622  0.22771  1.20580 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.895172   0.109110  35.699  < 2e-16 ***
s            0.092787   0.006666  13.920  < 2e-16 ***
expr         0.039344   0.006306   6.239 7.33e-10 ***
tenure       0.034209   0.007715   4.434 1.06e-05 ***
rns         -0.074532   0.028815  -2.587  0.00988 ** 
smsa         0.136737   0.027948   4.893 1.22e-06 ***
iq           0.003279   0.001083   3.028  0.00255 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3445 on 751 degrees of freedom
Multiple R-squared:   0.36,	Adjusted R-squared:  0.3548 
F-statistic: 70.39 on 6 and 751 DF,  p-value: < 2.2e-16


In [15]:
library(sandwich)
robust_se <- vcovHC(fit, type = "HC")

library(lmtest)
coeftest(fit, vcov = robust_se)


t test of coefficients:

              Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  3.8951718  0.1153921 33.7560 < 2.2e-16 ***
s            0.0927874  0.0069440 13.3623 < 2.2e-16 ***
expr         0.0393443  0.0066295  5.9347 4.494e-09 ***
tenure       0.0342090  0.0078591  4.3528 1.531e-05 ***
rns         -0.0745325  0.0298385 -2.4979  0.012707 *  
smsa         0.1367369  0.0276426  4.9466 9.325e-07 ***
iq           0.0032792  0.0011269  2.9099  0.003722 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


# 2SLS

In [16]:
install.packages("ivreg")

试开URL’https://cran.rstudio.com/bin/macosx/big-sur-arm64/contrib/4.4/ivreg_0.6-5.tgz'
Content type 'application/x-gzip' length 1220021 bytes (1.2 MB)
downloaded 1.2 MB




下载的二进制程序包在
	/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//RtmpnjHIuj/downloaded_packages里


In [19]:
library(ivreg)

fit_iv <- ivreg(
  lw ~ iq+s+expr+tenure+rns+smsa | s+ med+ kww+mrt+age+expr+tenure+rns+smsa,
  data = grilic
)
summary(fit_iv)


Call:
ivreg(formula = lw ~ iq + s + expr + tenure + rns + smsa | s + 
    med + kww + mrt + age + expr + tenure + rns + smsa, data = grilic)

Residuals:
       Min         1Q     Median         3Q        Max 
-1.3825405 -0.2437078  0.0009735  0.2514625  1.4609417 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.837875   0.346424  13.965  < 2e-16 ***
iq          -0.011547   0.005241  -2.203 0.027889 *  
s            0.137348   0.017042   8.059 3.02e-15 ***
expr         0.033804   0.007302   4.630 4.32e-06 ***
tenure       0.040564   0.008896   4.560 5.98e-06 ***
rns         -0.117698   0.035468  -3.318 0.000949 ***
smsa         0.149983   0.031572   4.751 2.43e-06 ***

Diagnostic tests:
                 df1 df2 statistic  p-value    
Weak instruments   4 748     10.54 2.61e-08 ***
Wu-Hausman         1 750     10.70  0.00112 ** 
Sargan             3  NA     61.14 3.36e-13 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual

In [22]:
install.packages("ivmodel")

还安装依赖关系‘reshape2’



试开URL’https://cran.rstudio.com/bin/macosx/big-sur-arm64/contrib/4.4/reshape2_1.4.4.tgz'
Content type 'application/x-gzip' length 332089 bytes (324 KB)
downloaded 324 KB

试开URL’https://cran.rstudio.com/bin/macosx/big-sur-arm64/contrib/4.4/ivmodel_1.9.1.tgz'
Content type 'application/x-gzip' length 423381 bytes (413 KB)
downloaded 413 KB




下载的二进制程序包在
	/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//RtmpnjHIuj/downloaded_packages里


In [23]:
library(ivmodel)

In [24]:
library(AER)

载入需要的程序包：car
载入需要的程序包：carData

载入程序包：‘car’

The following object is masked from ‘package:dplyr’:

    recode

The following object is masked from ‘package:purrr’:

    some

载入需要的程序包：survival
Registered S3 methods overwritten by 'AER':
  method              from 
  print.ivreg         ivreg
  print.summary.ivreg ivreg
  summary.ivreg       ivreg
  vcov.ivreg          ivreg
  bread.ivreg         ivreg
  estfun.ivreg        ivreg
  hatvalues.ivreg     ivreg
  predict.ivreg       ivreg
  anova.ivreg         ivreg
  terms.ivreg         ivreg
  model.matrix.ivreg  ivreg
  update.ivreg        ivreg

载入程序包：‘AER’

The following objects are masked from ‘package:ivreg’:

    ivreg, ivreg.fit



In [57]:
fit_iv <- AER::ivreg(
  lw ~ iq+s+expr+tenure+rns+smsa | s+ med+ kww+mrt+age+expr+tenure+rns+smsa,
  data = grilic,
  methods = "MM"
)
summary(fit_iv, diagnostics = TRUE)

警告信息:
1: In lm.fit(z, x, ...) :
 extra argument ‘methods’ will be disregarded 
2: In lm.fit(xz, y, offset = offset, ...) :
 extra argument ‘methods’ will be disregarded 



Call:
AER::ivreg(formula = lw ~ iq + s + expr + tenure + rns + smsa | 
    s + med + kww + mrt + age + expr + tenure + rns + smsa, data = grilic, 
    methods = "MM")

Residuals:
       Min         1Q     Median         3Q        Max 
-1.3825405 -0.2437078  0.0009735  0.2514625  1.4609417 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.837875   0.346424  13.965  < 2e-16 ***
iq          -0.011547   0.005241  -2.203 0.027889 *  
s            0.137348   0.017042   8.059 3.02e-15 ***
expr         0.033804   0.007302   4.630 4.32e-06 ***
tenure       0.040564   0.008896   4.560 5.98e-06 ***
rns         -0.117698   0.035468  -3.318 0.000949 ***
smsa         0.149983   0.031572   4.751 2.43e-06 ***

Diagnostic tests:
                 df1 df2 statistic  p-value    
Weak instruments   4 748     10.54 2.61e-08 ***
Wu-Hausman         1 750     10.70  0.00112 ** 
Sargan             3  NA     61.14 3.36e-13 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.

In [31]:
result <- summary(fit_iv, diagnostics =TRUE)

In [32]:
result$diagnostics

                 df1 df2 statistic      p-value
Weak instruments   4 748  10.53826 2.608750e-08
Wu-Hausman         1 750  10.69904 1.120907e-03
Sargan             3  NA  61.13660 3.360347e-13

# GMM

In [60]:
library(ivreg)

fit_iv <- ivreg(
  lw ~ iq+s+expr+tenure+rns+smsa | s+ med+ kww+mrt+age+expr+tenure+rns+smsa,
  data = grilic
)
summary(fit_iv)


Call:
ivreg(formula = lw ~ iq + s + expr + tenure + rns + smsa | s + 
    med + kww + mrt + age + expr + tenure + rns + smsa, data = grilic, 
    method = "qr")

Residuals:
       Min         1Q     Median         3Q        Max 
-1.3825405 -0.2437078  0.0009735  0.2514625  1.4609417 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.837875   0.346424  13.965  < 2e-16 ***
iq          -0.011547   0.005241  -2.203 0.027889 *  
s            0.137348   0.017042   8.059 3.02e-15 ***
expr         0.033804   0.007302   4.630 4.32e-06 ***
tenure       0.040564   0.008896   4.560 5.98e-06 ***
rns         -0.117698   0.035468  -3.318 0.000949 ***
smsa         0.149983   0.031572   4.751 2.43e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3851 on 751 degrees of freedom
Multiple R-Squared: 0.2002,	Adjusted R-squared: 0.1938 
Wald test: 55.92 on 6 and 751 DF,  p-value: < 2.2e-16 


In [36]:
install.packages("gmm")

试开URL’https://cran.rstudio.com/bin/macosx/big-sur-arm64/contrib/4.4/gmm_1.8.tgz'
Content type 'application/x-gzip' length 1438237 bytes (1.4 MB)
downloaded 1.4 MB




下载的二进制程序包在
	/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//RtmpnjHIuj/downloaded_packages里


In [61]:
# 使用广义矩估计
library(gmm)

# 定义矩条件函数
g <- function(theta, data) {
  y <- data$lw
  x <- cbind(
    1, data$s, data$iq, data$expr, data$tenure,
    data$rns, data$smsa
  ) # 内生和外生变量
  z <- cbind(
    data$s, data$med, data$kww, data$mrt, data$age,
    data$expr, data$tenure, data$rns, data$smsa
  ) # 工具变量
  
  beta <- theta[1:7] # 参数向量
  e <- y - x %*% beta # 残差
  
  g <- z %*% t(e) # 矩条件
  
  return(as.vector(g))
}

# 使用0作为初始猜测
init_theta <- coef(
  lm(lw ~ s + iq + expr + tenure + rns + smsa, data = grilic)
)

# 执行gmm
gmm <- gmm(
  g = g,
  x = grilic,
  t0 = init_theta,
  type = "twoStep" # 可以选择 "twoStep", "iterative", 或 "cue"
  )
  # 查看结果
  summary(gmm)

: [1m[33mError[39m in `z %*% t(e)`:[22m
[33m![39m 非整合参数

In [56]:
library(gmm)

# 定义正确的矩条件函数
g <- function(theta, data) {
  # 提取变量
  y <- data$lw
  # 解释变量矩阵 (包括截距项)
  x <- with(data, cbind(1, s, iq, expr, tenure, rns, smsa))  # 假设iq是内生变量
  # 工具变量矩阵 (所有外生变量 + 工具变量，排除内生变量)
  z <- with(data, cbind(1, s, med, kww, mrt, age, expr, tenure, rns, smsa))  # 确保外生
  
  # 提取参数
  beta <- theta[1:ncol(x)]  # 参数与x的列数一致
  
  # 计算残差
  e <- y - x %*% beta
  
  # 矩条件：每个观测的 e_i * z_i，返回 n × q 矩阵
  return(e * z)
}

# 设置初始值（需与参数数量一致）
init_theta <- rep(0, 7)  # 初始化为0向量，或使用OLS结果
# 若用OLS初始值：
# init_theta <- coef(lm(lw ~ s + iq + expr + tenure + rns + smsa, data = grilic))

# 执行GMM估计
gmm_fit <- gmm(
  g = g,
  x = grilic,  # 确保数据框名称正确
  t0 = init_theta,
  type = "twoStep",  # 推荐使用两步GMM
  # 可选：添加优化控制参数
  control = list(maxit = 1000, reltol = 1e-6)
)

# 查看结果
summary(gmm_fit)

: [1m[33mError[39m in `e * z`:[22m
[33m![39m 非整合陈列

In [47]:
install.packages("plm")

还安装依赖关系‘miscTools’, ‘bdsmatrix’, ‘collapse’, ‘maxLik’



试开URL’https://cran.rstudio.com/bin/macosx/big-sur-arm64/contrib/4.4/miscTools_0.6-28.tgz'
Content type 'application/x-gzip' length 89857 bytes (87 KB)
downloaded 87 KB

试开URL’https://cran.rstudio.com/bin/macosx/big-sur-arm64/contrib/4.4/bdsmatrix_1.3-7.tgz'
Content type 'application/x-gzip' length 319669 bytes (312 KB)
downloaded 312 KB

试开URL’https://cran.rstudio.com/bin/macosx/big-sur-arm64/contrib/4.4/collapse_2.0.19.tgz'
Content type 'application/x-gzip' length 7998817 bytes (7.6 MB)
downloaded 7.2 MB

Error in download.file(url, destfile, method, mode = "wb", ...) : 
  download from 'https://cran.rstudio.com/bin/macosx/big-sur-arm64/contrib/4.4/collapse_2.0.19.tgz' failed
此外: 警告信息:
1: In download.file(url, destfile, method, mode = "wb", ...) :
  downloaded length 7593842 != reported length 7998817
2: In download.file(url, destfile, method, mode = "wb", ...) :
  URL 'https://cran.rstudio.com/bin/macosx/big-sur-arm64/contrib/4.4/collapse_2.0.19.tgz': Timeout of 60 seconds was reache


下载的二进制程序包在
	/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//RtmpnjHIuj/downloaded_packages里


In [50]:
library(plm)


载入程序包：‘plm’

The following objects are masked from ‘package:dplyr’:

    between, lag, lead



In [51]:
help(plm)

In [67]:
library(gmm)

# 假设 grilic 数据集已经加载
# 定义自变量矩阵 X 和工具变量矩阵 Z
X <- cbind(
  1, grilic$iq, grilic$s, grilic$expr, grilic$tenure, 
  grilic$rns, grilic$smsa
)  # 自变量矩阵 X
Z <- cbind(
  grilic$s, grilic$med, grilic$kww, grilic$mrt, grilic$age, 
  grilic$expr, grilic$tenure, grilic$rns, grilic$smsa
)  # 工具变量矩阵 Z
lw <- grilic$lw  # 因变量

# 定义 GMM 矩条件函数
gmm_model <- function(theta, data) {
  # 提取数据
  X <- data$X
  Z <- data$Z
  lw <- data$lw
  
  # 计算残差
  epsilon <- lw - X %*% theta  # 残差计算: lw - X * beta
  
  # 计算矩条件: Z' * epsilon
  moment_conditions <- t(Z) %*% epsilon
  
  return(moment_conditions)
}

# 使用 gmm 包进行估计
# t0 是初始参数估计值，这里设为零向量
fit_gmm <- gmm(gmm_model, x = list(X = X, Z = Z, lw = lw), t0 = rep(0, ncol(X)))

# 输出标准 GMM 结果
summary(fit_gmm)


警告信息:
In FinRes.baseGmm.res(z, Model_info) :
  The covariance matrix of the coefficients is singular


In [64]:
grilic

# A tibble: 758 × 20
     rns rns80   mrt mrt80  smsa smsa80   med    iq   kww  year   age age80
   <dbl> <dbl> <dbl> <dbl> <dbl>  <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1     0     0     0     1     1      1     8    93    35    68    19    31
 2     0     0     0     1     1      1    14   119    41    66    23    37
 3     0     0     0     1     1      1    14   108    46    67    20    33
 4     0     0     0     1     1      1    12    96    32    66    18    32
 5     0     0     1     1     1      1     6    74    27    73    26    34
 6     0     0     0     0     1      1     8    91    24    66    16    30
 7     0     0     1     1     1      1     8   114    50    73    30    38
 8     0     0     1     1     0      0    14   111    37    67    23    36
 9     0     0     1     1     1      1    12    95    44    66    22    36
10     0     0     1     1     0      0    13   132    44    73    30    38
# ℹ 748 more rows
# ℹ 8 more variables: s <dbl>, s80 <dbl>, expr <d

In [70]:
library(gmm)
library(AER)

# 示例数据集 grilic 已经加载，假设包含变量 lw, iq, s, expr, tenure, rns, smsa, med, kww, mrt, age

# 使用 ivreg 估计 OLS 工具变量回归模型
ivreg_fit <- ivreg(lw ~ iq + s + expr + tenure + rns + smsa | s + med + kww + mrt + age + expr + tenure + rns + smsa, data = grilic)

# 提取 ivreg 结果中的系数作为初始估计
beta_ivreg <- coef(ivreg_fit)

# 定义 GMM 矩条件函数
gmm_model <- function(theta, data) {
  # 提取数据
  X <- data$X
  Z <- data$Z
  lw <- data$lw
  
  # 计算残差
  epsilon <- lw - X %*% theta  # 残差计算: lw - X * beta
  
  # 计算矩条件: Z' * epsilon
  moment_conditions <- Z %*% epsilon
  
  return(moment_conditions)
}

# 准备 GMM 数据（自变量 X 和工具变量 Z）
X <- cbind(1, grilic$iq, grilic$s, grilic$expr, grilic$tenure, grilic$rns, grilic$smsa)  # 自变量矩阵 X
Z <- cbind(grilic$s, grilic$med, grilic$kww, grilic$mrt, grilic$age, grilic$expr, grilic$tenure, grilic$rns, grilic$smsa)  # 工具变量矩阵 Z
lw <- grilic$lw  # 因变量

# 将数据传递给 GMM
gmm_data <- list(X = X, Z = Z, lw = lw)

# 使用 GMM 方法进行估计
fit_gmm <- gmm(gmm_model, x = gmm_data, t0 = beta_ivreg)

# 输出 GMM 结果
summary(fit_gmm)

: [1m[33mError[39m in `Z %*% epsilon`:[22m
[33m![39m 非整合参数

In [72]:
library(gmm)

# 定义矩条件函数
g <- function(theta, data) {
  y <- data$lw
  x <- cbind(1, data$s, data$iq, data$expr, data$tenure, data$rns, data$smsa) # 内生和外生变量
  z <- cbind(data$s, data$med, data$kww, data$expr, data$tenure, data$rns, data$smsa) # 工具变量
  
  beta <- theta[1:7] # 参数向量
  e <- as.vector(y - x %*% beta) # 计算残差
  
  g <- as.vector(t(z) %*% e) # 使用矩阵乘法计算矩条件，并将其转换为向量
  return(g)
}

# 提供初始参数猜测值
init_theta <- coef(lm(lw ~ s + iq + expr + tenure + rns + smsa, data = grilic))

# 执行 GMM 估计
res_gmm <- gmm(
  g = g,
  x = grilic,
  t0 = init_theta,
  type = "twoStep" # 可以选择 "twoStep", "iterative", 或 "cue"
)

# 查看结果
summary(res_gmm)

警告信息:
In FinRes.baseGmm.res(z, Model_info) :
  The covariance matrix of the coefficients is singular



Call:
gmm(g = g, x = grilic, t0 = init_theta, type = "twoStep")


Method:  twoStep 

Kernel:  Quadratic Spectral(with bw =  0.65032 )

Coefficients:
             Estimate    Std. Error  t value     Pr(>|t|)  
(Intercept)   3.9109801         Inf   0.0000000   1.0000000
s             0.0910978         Inf   0.0000000   1.0000000
iq            0.0026866         Inf   0.0000000   1.0000000
expr          0.0570079         Inf   0.0000000   1.0000000
tenure        0.0339528         Inf   0.0000000   1.0000000
rns          -0.0095459         Inf   0.0000000   1.0000000
smsa          0.1692517         Inf   0.0000000   1.0000000

J-Test: degrees of freedom is -6 
                J-test                P-value             
Test E(g)=0:    2.08055235121342e-08  *******             

Initial values of the coefficients
 (Intercept)            s           iq         expr       tenure          rns 
 3.910980111  0.091097767  0.002686621  0.057007851  0.033952826 -0.009545867 
        smsa 
 0.169251

In [73]:
# 定义矩条件函数
g <- function(theta, data) {
  y <- data$lw
  x <- cbind(1, data$s, data$iq, data$expr, data$tenure, data$rns, data$smsa) # 内生和外生变量
  z <- cbind(data$s, data$med, data$kww) # 工具变量
  
  beta <- theta[1:7] # 参数向量
  e <- as.vector(y - x %*% beta) # 计算残差
  
  g <- as.vector(crossprod(z, e)) # 使用crossprod计算矩条件，并将其转换为向量
  return(g)
}

# 提供初始参数猜测值
init_theta <- coef(lm(lw ~ s + iq + expr + tenure + rns + smsa, data = grilic))

# 执行 GMM 估计
res_gmm <- gmm(
  g = g,
  x = grilic,
  t0 = init_theta,
  type = "twoStep" # 可以选择 "twoStep", "iterative", 或 "cue"
)

# 查看结果
summary(res_gmm)

警告信息:
In ar.ols(x, aic = aic, order.max = order.max, na.action = na.action,  :
  model order:  1 singularities in the computation of the projection matrix results are only valid up to model order 0


: [1m[33mError[39m in `AllArg$bw()`:[22m
[33m![39m Cannot compute bandwidth because AR(1) approximation failed for estimating functions of: 

In [75]:
# 定义目标函数
model_gmm <- function(theta, data) {
  # 设定回归方程：y = X * beta + epsilon
  lw <- data$lw
  s <- data$s
  expr <- data$expr
  tenure <- data$tenure
  rns <- data$rns
  smsa <- data$smsa
  iq <- data$iq
  med <- data$med
  kww <- data$kww
  
  # X是自变量，y是因变量
  X <- cbind(s, expr, tenure, rns, smsa)
  Z <- cbind(med, kww)  # 工具变量
  
  # 模型中使用工具变量的回归：计算残差
  residuals <- lw - X %*% theta
  
  # GMM目标函数：工具变量和残差的矩阵乘积
  moment_conditions <- colSums(Z * residuals)
  return(moment_conditions)
}

# GMM估计
initial_theta <- rep(0, 5)  # 初始参数值
gmm_result <- gmm(model_gmm, x = data, t0 = initial_theta)

# 查看结果
summary(gmm_result)


: [1m[33mError[39m in `data$lw`:[22m
[33m![39m 类别为'closure'的对象不可以取子集